In [ ]:
from fastai.vision.all import *

In [ ]:
pd.options.display.max_columns = 100

In [ ]:
datapath = Path("../rsna_data/")
train_df = pd.read_csv(datapath/'train.csv')
train_df.pe_present_on_image.mean()

In [ ]:
do_sample=False
if do_sample:
    sample_pids_dir = (datapath/'sample_pids')
    valid_pids = pd.read_pickle(sample_pids_dir/'valid_pids.pkl')
    train_pids = pd.read_pickle(sample_pids_dir/'train_pids.pkl')

    len(train_pids), len(valid_pids)

In [ ]:
do_cv = True
FOLD = 0

if do_cv: 
    cv_pids_dir = (datapath/'cv_pids')
    if not cv_pids_dir.exists(): cv_pids_dir.mkdir()
    cv_df = train_df[['StudyInstanceUID', 'negative_exam_for_pe']].drop_duplicates().reset_index(drop=True)
    all_pids = cv_df['StudyInstanceUID'].values

    valid_pids = pd.read_pickle(datapath/f'cv_pids/pids_fold{FOLD}.pkl')
    train_pids = list(set(all_pids).difference(valid_pids))

In [ ]:
len(train_pids), len(valid_pids), len(train_pids+valid_pids)

In [ ]:
metadata_path = datapath/'metadata'
train_metadata_paths = [o for o in metadata_path.ls() if o.stem in train_pids]
valid_metadata_paths = [o for o in metadata_path.ls() if o.stem in valid_pids]

In [ ]:
len(train_metadata_paths), len(valid_metadata_paths)

### Baseline valid performance

In [ ]:
train_metadf = pd.concat([pd.read_csv(o) for o in train_metadata_paths]).reset_index(drop=True)
valid_metadf = pd.concat([pd.read_csv(o) for o in valid_metadata_paths]).reset_index(drop=True)

In [ ]:
train_metadf.shape, valid_metadf.shape

In [ ]:
train_metadf = train_metadf.merge(train_df, on=['StudyInstanceUID', 'SOPInstanceUID'])
valid_metadf = valid_metadf.merge(train_df, on=['StudyInstanceUID', 'SOPInstanceUID'])

In [ ]:
train_metadf['pe_present_on_image'].mean(), valid_metadf['pe_present_on_image'].mean()

In [ ]:
p = valid_metadf['pe_present_on_image'].mean()

In [ ]:
-(p*np.log(p)+(1-p)*np.log(1-p))

In [ ]:
targs = tensor(valid_metadf['pe_present_on_image'].values)
preds = torch.ones_like(targs)*p

In [ ]:
F.binary_cross_entropy(preds,targs.float())

In [ ]:
preds = torch.stack([torch.ones_like(targs)*(1-p), torch.ones_like(targs)*(p)],1)

In [ ]:
preds

In [ ]:
baseline_ce = F.nll_loss(torch.log(preds), targs); baseline_ce

In [ ]:
baseline_acc = 1 - p; baseline_acc

In [ ]:
(1-p)/p

### All samples

In [ ]:
labels_dict = dict(zip(train_df['SOPInstanceUID'], train_df['pe_present_on_image']))

In [ ]:
len(labels_dict), len(train_df)

In [ ]:
imgdatapath = (datapath/'full_raw_512')

In [ ]:
files = get_image_files(imgdatapath)

In [ ]:
train_files = [o for o in files if o.parent.name in train_pids]
valid_files = [o for o in files if o.parent.name in valid_pids]

In [ ]:
len(train_files), len(valid_files)

In [ ]:
len(files)

### Smart Sample

We don't have to use all the slices as input data per patient. We can simply sample every nth slice for each patient so that we have good enough variability within that patient and use that data for CNN feature training. But always keep, if y = 1.

In [ ]:
from fastai.medical.imaging import *

In [ ]:
files_dict = defaultdict(list)
for o in files:
    files_dict[o.parent.name].append(o)

In [ ]:
for k in files_dict:
    files_dict[k] = sorted(files_dict[k], key=lambda o: int(o.name.split('_')[0]))

In [ ]:
nslices = [len(files_dict[k]) for k in files_dict]
plt.hist(nslices)

In [ ]:
def sample_patient_slices(pid, num_slice_samples):
    "Use a fixed number of samples per patient for training speed up"
    files = array(files_dict[pid])
    n = len(files)
    if n > num_slice_samples:
        labels = [labels_dict[o.stem.split("_")[1]] for o in files]
        pos_idxs = list(np.where(labels)[0])
        idxs = [np.clip(int(i), 0, n-1) for i in np.linspace(0, n, num_slice_samples)]
        idxs += pos_idxs
        idxs = sorted(set(idxs))
        return files[idxs]
    else: return files

In [ ]:
train_sampled_files = parallel(partial(sample_patient_slices, num_slice_samples=200), train_pids)

In [ ]:
train_files = []
for o in train_sampled_files: train_files += list(o)

In [ ]:
valid_files = []
for o in valid_pids: valid_files += files_dict[o]

In [ ]:
len(train_files), len(valid_files)

### Data

In [ ]:
def aug_transforms(mult=1.0, do_flip=True, flip_vert=False, max_rotate=10., min_zoom=1., max_zoom=1.1,
                   max_lighting=0.2, max_warp=0.2, p_affine=0.75, p_lighting=0.75, xtra_tfms=[], size=None,
                   mode='bilinear', pad_mode=PadMode.Reflection, align_corners=True, batch=False, min_scale=1.):
    "Utility func to easily create a list of flip, rotate, zoom, warp, lighting transforms."
    res,tkw = [],dict(size=size if min_scale==1. else None, mode=mode, pad_mode=pad_mode, batch=batch, align_corners=align_corners)
    max_rotate,max_lighting,max_warp = array([max_rotate,max_lighting,max_warp])*mult
    if do_flip: res.append(Dihedral(p=0.5, **tkw) if flip_vert else Flip(p=0.5, **tkw))
    if max_warp:   res.append(Warp(magnitude=max_warp, p=p_affine, **tkw))
    if max_rotate: res.append(Rotate(max_deg=max_rotate, p=p_affine, **tkw))
    if min_zoom<1 or max_zoom>1: res.append(Zoom(min_zoom=min_zoom, max_zoom=max_zoom, p=p_affine, **tkw))
    if max_lighting:
        res.append(Brightness(max_lighting=max_lighting, p=p_lighting, batch=batch))
        res.append(Contrast(max_lighting=max_lighting, p=p_lighting, batch=batch))
    if size: xtra_tfms += [RandomResizedCropGPU(size, min_scale=min_scale, ratio=(1,1))]
    return res + L(xtra_tfms)

In [ ]:
wgtdict = {0:1, 1:10}

In [ ]:
def get_label(o): 
    return labels_dict[o.stem.split("_")[1]]

In [ ]:
class FlipUD(RandTransform):
    def __init__(self, p=0.5): super().__init__(p=p)
    def encodes(self, x:TensorImage): return x.flip(-2)

In [ ]:
from time import time
def get_dls(train_files, valid_files, resize=256, size=224, bs=128):
    
    files = train_files + valid_files
    trn_wgts = [wgtdict[get_label(o)] for o in train_files]
    print(f"Collected idxs")

    tfms = [[PILImage.create, ToTensor, RandomResizedCrop(resize, min_scale=0.9)], 
            [get_label, Categorize()]]
    dsets = Datasets(files, tfms=tfms,
                     splits=FuncSplitter(lambda o: True if o.parent.name in valid_pids else False)(files))
    print(f"Created dset")

    aug_tfms = aug_transforms(size=size, max_lighting=False, max_warp=False, flip_vert=False, min_scale=0.75,
                              xtra_tfms=[
                                      RandomErasing(p=0.25, sh=0.2, min_aspect=0.15), 
                                        ])    
    batch_tfms = [IntToFloatTensor] + aug_tfms
    dls = dsets.dataloaders(bs=bs, after_batch=batch_tfms, dl_type=WeightedDL, dl_kwargs=[{"wgts":trn_wgts}, {}])
    print(f"DLs ready")
    return dls

In [ ]:
opt_func = partial(ranger, **dict(sqrmom=0.99, mom=0.95, beta=0., eps=1e-4))

In [ ]:
# Progressive Resizing
resize, size = (512, 480)
# resize, size = (256, 224)
dls = get_dls(train_files, valid_files, resize=resize, size=size, 
#               bs=int(64*(256/resize))
              bs=32
             )

In [ ]:
resize, size

In [ ]:
loss_func = LabelSmoothingCrossEntropyFlat(eps=0.01)

In [ ]:
# learn = cnn_learner(dls, xresnet34, opt_func=opt_func, 
#                     pretrained=False, loss_func=loss_func,
#                     metrics=[accuracy], cbs=[SaveModelCallback("accuracy", 
#                                                                fname=f"xresnet34-{resize}-PR-fold{FOLD}")])

In [ ]:
from efficientnet_pytorch import EfficientNet
model = EfficientNet.from_name('efficientnet-b3', num_classes=2)
learn = Learner(dls, model, opt_func=opt_func, 
                    pretrained=False, loss_func=loss_func,
                    metrics=[accuracy], cbs=[SaveModelCallback("accuracy", 
                                                               fname=f"effb3-{resize}-PR-fold{FOLD}")])

In [ ]:
learn.dls.bs

In [ ]:
loadpth = Path(f"effb3-256-PR-fold{FOLD}")
learn.load(loadpth)
print("loaded...")

In [ ]:
learn.to_fp16();

In [ ]:
learn.fit_flat_cos(3, 0.01)

In [ ]:
# learn.export(f"./models/xresnet34-{resize}-PR-fold{FOLD}-export.pkl");
learn.export(f"./models/effb3-{resize}-PR-fold{FOLD}-export.pkl");

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
interp.plot_confusion_matrix()

In [ ]:
interp.plot_top_losses(20)

In [ ]:
interp.print_classification_report()

In [ ]:
learn.show_results(max_n=16)

In [ ]:
xb,yb=learn.dls.train.one_batch()
yb.float().mean()

In [ ]:
plt.hist(interp.preds[:,1].numpy())

### Generate OOF Preds & Embeddings